<a href="https://colab.research.google.com/github/mille-s/Build_KGs_entities/blob/main/LongInput_D2T_25_EvalLLM_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install Aixplain (needed to load pickle files)
from IPython.display import clear_output
import os
! pip install aixplain
clear_output()

! pip install diversity

os.environ["AIXPLAIN_API_KEY"] = '3d039057dcf4e221c7c722debaf56d05d33a565438db0b04b5ead21dac4449de'

# Calculate system-level scores LLM-as-judge
Code taken and adapted from the GEM 2024 code

In [ ]:
#@title Unzip uploaded file
import os
import zipfile
import glob
import re
from IPython.display import clear_output

# !unzip /content/GPT_results.zip -d /content
zip_language = 'GA' #@param['EN', 'GA']
# Claude2 is for the second evaluation of long texts
zip_model = 'Claude2' #@param['GPT-o3', 'Claude', 'Claude2', 'Llama-13B', 'R1_Llama-70B', 'R1_Llama-70B_bis']
zip_data = 'regular' #@param['regular', 'iaa']
zip_input_percentage = '100' #@param['100', '85to100']

def unzipEvals(path_dir, zip_path):
  if os.path.exists(path_dir):
    # Remove folder contents
    for filename in os.listdir(path_dir):
      file_path = os.path.join(path_dir, filename)
  else:
    os.makedirs(path_dir)
    print(f'Created folder {path_dir}')
  ! unzip {zip_path} -d {path_dir}
  print(f'Unzipped {zip_path} to {path_dir}')

path_dir = os.path.join(zip_language+'_'+zip_data, zip_model)
zip_path = ''
if zip_data == 'regular':
  if zip_model == 'GPT-o3':
    if zip_language == 'EN':
      zip_path = '/content/EN_GPT_results.zip'
    elif zip_language == 'GA':
      zip_path = '/content/GA_GPT_results.zip'
  elif zip_model == 'Claude':
    if zip_language == 'EN':
      zip_path = '/content/EN_Claude_results.zip'
    elif zip_language == 'GA':
      zip_path = '/content/GA_Claude_results.zip'
  elif zip_model == 'Claude2':
    if zip_language == 'EN':
      zip_path = f'/content/en_longInputD2T_LLMtexts_inputs{zip_input_percentage}.zip'
    elif zip_language == 'GA':
      zip_path = f'/content/ga_longInputD2T_LLMtexts_inputs{zip_input_percentage}.zip'
  elif zip_model == 'Llama-13B':
    if zip_language == 'EN':
      zip_path = '/content/EN_aiXplainLlama_results.zip'
    elif zip_language == 'GA':
      zip_path = '/content/GA_aiXplainLlama_results.zip'
  elif zip_model == 'R1_Llama-70B':
    if zip_language == 'EN':
      zip_path = '/content/EN_aiXplainDeepseek_results.zip'
    elif zip_language == 'GA':
      zip_path = '/content/GA_aiXplainDeepseek_results.zip'
  elif zip_model == 'R1_Llama-70B_bis':
    if zip_language == 'EN':
      zip_path = '/content/EN_aiXplainDeepseek_results2.zip'
    elif zip_language == 'GA':
      zip_path = ''

unzipEvals(path_dir, zip_path)

#title Rename files in en_longInputD2T_LLMtexts_inputs85to100
if os.path.exists(f'/content/{zip_language}_regular/Claude2/{zip_language.lower()}_longInputD2T_LLMtexts_inputs85to100'):
  list_filepaths = glob.glob(f'/content/{zip_language}_regular/Claude2/{zip_language.lower()}_longInputD2T_LLMtexts_inputs85to100/*.pkl')
  for filepath in list_filepaths:
    head, tail = os.path.split(filepath)
    fullname = os.path.splitext(tail)[0]
    extension = os.path.splitext(tail)[1]
    # remove system ID
    basename = fullname.rsplit('_', 1)[0]
    new_fullname = ''
    if re.search('inputs100', fullname):
      new_fullname = basename.replace('inputs100', 'inputs85to100')+'_0'+extension
    if re.search('inputs95', fullname):
      new_fullname = basename.replace('inputs95', 'inputs85to100')+'_6'+extension
    elif re.search('inputs90', fullname):
      new_fullname = basename.replace('inputs90', 'inputs85to100')+'_7'+extension
    elif re.search('inputs85', fullname):
      new_fullname = basename.replace('inputs85', 'inputs85to100')+'_8'+extension
    os.rename(filepath, os.path.join(head, new_fullname))

# clear_output()

In [ ]:
#@title Load unzipped files and save in normalised json files
import pickle
import glob
import json
import re
import codecs

def is_valid_pickle(filepath):
  try:
    with open(filepath, 'rb') as f:
      # Check header
      header = f.read(1)
      if header != b'\x80':
          return False
      f.seek(0)
      # Attempt to load safely
      pickle.load(f)
    return True
  except Exception:
    return False


update_params_unzip = True #@param {type:"boolean"}
if update_params_unzip:
  zip_language = 'GA' #@param['EN', 'GA']
  zip_model = 'Claude2' #@param['GPT-o3', 'Claude', 'Claude2', 'Llama-13B', 'R1_Llama-70B', 'R1_Llama-70B_bis']
  zip_data = 'regular' #@param['regular', 'iaa']

model_scores = [1, 2, 3, 4, 5, 6, 7]
# load_gemini_folder = True #@param {type:"boolean"}
# load_gpt_folder = False #@param {type:"boolean"}
path_dir_unzipped = ''
model_prefix = ''
if zip_model.startswith('Claude_bis'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, zip_language+'_Claude_not_aiXplainLlama_results')
  model_prefix = 'claude-3-7-sonnet-latest'
elif zip_model.startswith('Claude2'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, zip_language.lower()+'_longInputD2T_LLMtexts_inputs'+zip_input_percentage)
  model_prefix = 'claude_anthropic'
elif zip_model.startswith('Claude'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, zip_language+'_Claude_results')
  model_prefix = 'claude-3-7-sonnet-latest'
elif zip_model.startswith('GPT'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, zip_language+'_GPT_results')
  model_prefix = 'o3'
elif zip_model.startswith('Llama'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, zip_language+'_aiXplainLlama_results')
  # ERROR HERE? The Llama file actually has that prefix inside
  model_prefix = '6704c91bfdf7d14548c9fedb'
elif zip_model.startswith('R1_Llama-70B_bis'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, zip_language+'_aiXplainDeepseek_results2')
  model_prefix = '67976f47e341d313f66bb835'
elif zip_model.startswith('R1_Llama-70B'):
  path_dir_unzipped = os.path.join('/content', zip_language+'_'+zip_data, zip_model, zip_language+'_aiXplainDeepseek_results')
  model_prefix = '67976f47e341d313f66bb835'

def separateJustification(LLMoutString, criterion):
  """
  The Justifications returned by the models often break the json format, so I extract them
  """
  search_expression = '("'+criterion+'":[^\{]+\{[^\}]*"Justification":)([^\}]+)("Score":[^\}]+\})'
  if re.search(search_expression, LLMoutString):
    justificationRemoved = re.sub(search_expression, '\g<1> "", \g<3>',  LLMoutString)
    justification = re.sub('^.*'+search_expression+'.*$', '\g<2>',  LLMoutString)
  else:
    justificationRemoved = LLMoutString
    justification = ''
  return justificationRemoved, justification

def loadDataPoint(dbfile_x, model, sys0_to_update):
  eval_missing = None
  wrong_score = None
  dico_key = 'scores_'+str(model)
  formatted_scores = {}
  # load data with pickle
  dp = {}
  if is_valid_pickle(dbfile_x.name):
    dp = pickle.load(dbfile_x)
  else:
    print('The input file is not a pickle file!')
  print(f'\nHere is the full dico with ratings:\n{dp}')
  if dico_key in dp:
    print(f"\nID found: {dp['id']}")
    print(f"Triples found: {dp['triples']}")
    print(f"Text found: {dp['text']}")
    justifications = []
    # pickle.load uses single quotes, whereas json.load expects double quotes
    # Gemini adds a node "query" in the json, unlike OpenAI's models
    LLMout_string = str(dp[dico_key]).replace("'query'", '"query"')
    LLMout_string = LLMout_string.replace("```json", "")
    LLMout_string = LLMout_string.replace("```", "")
    # DeepSeek output is quite different from the rest
    if model == '67976f47e341d313f66bb835':
      LLMout_string = LLMout_string.replace("\\n", '')
      LLMout_string = LLMout_string.replace("\\", '')
      LLMout_string = LLMout_string.replace('"', "'")
      # Can't rely on double bracket closing because sometimes there's only one.
      LLMout_string = re.sub("^.*('output': '\{'No-Omissions': \{[^\}]+\}, 'No-Additions': \{[^\}]+\}, 'Grammaticality': \{[^\}]+\}, 'Fluency': \{[^\}]+\}).*$", "\g<1>", LLMout_string)
    LLMout_string = LLMout_string.replace("'No-Omissions'", '"No-Omissions"')
    # There's a typo in one of the Gemini outputs
    LLMout_string = LLMout_string.replace("'No-Omissons'", '"No-Omissions"')
    LLMout_string = LLMout_string.replace('"No-Omissons"', '"No-Omissions"')
    LLMout_string = LLMout_string.replace("'No-Additions'", '"No-Additions"')
    LLMout_string = LLMout_string.replace("'Grammaticality'", '"Grammaticality"')
    LLMout_string = LLMout_string.replace("'Fluency'", '"Fluency"')
    # Sometimes justifications are followed by single quotes, sometimes by double quotes
    LLMout_string = LLMout_string.replace("'Justification': '", '"Justification": "').replace("'Justification'", '"Justification"')
    LLMout_string = LLMout_string.replace("', 'Score'", '", "Score"').replace("'Score'", '"Score"')
    # print(LLMout_string)
    if LLMout_string == 'None':
      eval_missing = dp['id']
    else:
      # print(LLMout_string)
      # LLMout_string = re.sub('("No-Omissions":[^\{]+\{"Justification":)([^\}]+)("Score":[^\}]+\})', '\g<1> "", \g<3>',  LLMout_string)
      # Claude adds some comments before giving the eval results, this breaks the format
      LLMout_string = re.sub("^[^\{]+", "", LLMout_string)
      LLMout_string, justifNoOm = separateJustification(LLMout_string, 'No-Omissions')
      justifications.append(justifNoOm)
      LLMout_string, justifNoAdd = separateJustification(LLMout_string, 'No-Additions')
      justifications.append(justifNoAdd)
      LLMout_string, justifGram = separateJustification(LLMout_string, 'Grammaticality')
      justifications.append(justifGram)
      LLMout_string, justifFlu = separateJustification(LLMout_string, 'Fluency')
      justifications.append(justifFlu)
      print(f"Justifications found: {justifications}")
      LLMout_string = LLMout_string.replace("'1'", '"1"')
      LLMout_string = LLMout_string.replace("'2'", '"2"')
      LLMout_string = LLMout_string.replace("'3'", '"3"')
      LLMout_string = LLMout_string.replace("'4'", '"4"')
      LLMout_string = LLMout_string.replace("'5'", '"5"')
      LLMout_string = LLMout_string.replace("'6'", '"6"')
      LLMout_string = LLMout_string.replace("'7'", '"7"')
      # LLMout_string should finish with 2 braces (cases with only one have been seen)
      if LLMout_string.count('{') > LLMout_string.count('}'):
        while not re.search('\}[\s\r\n]*\}$', LLMout_string):
          LLMout_string = LLMout_string + ' }'
      # print(f'LLMOUT: <<<<<< {LLMout_string}>>>>>>')
      scores_json = json.loads(LLMout_string)
      clean_scores_json = None
      # Gemini adds a node "query" in the json, unlike OpenAI's models
      if 'query' in scores_json:
        clean_scores_json = scores_json['query']
      else:
        clean_scores_json = scores_json

      gram_score = int(clean_scores_json['Grammaticality']['Score'])
      flu_score = int(clean_scores_json['Fluency']['Score'])
      no_om_score = int(clean_scores_json['No-Omissions']['Score'])
      no_ad_score = int(clean_scores_json['No-Additions']['Score'])
      coh_score = 0
      if zip_model == 'Claude2':
        coh_score = int(clean_scores_json['Coherence']['Score'])
        if coh_score not in model_scores:
          wrong_score = dp['id']

      if (gram_score not in model_scores) or (flu_score not in model_scores) or (no_om_score not in model_scores) or (no_ad_score not in model_scores):
        wrong_score = dp['id']

      id_to_use = dp['id']
      if not sys0_to_update == None:
        id_to_use = dp['id'].rsplit('_', 1)[0]+f'_{sys0_to_update}'
      formatted_scores["eid"] = id_to_use
      formatted_scores["annotator_id"] = str(zip_model)
      formatted_scores["no-omissions"] = no_om_score
      formatted_scores["no-additions"] = no_ad_score
      formatted_scores["grammaticality"] = gram_score
      formatted_scores["fluency"] = flu_score
      if zip_model == 'Claude2':
        formatted_scores["coherence"] = coh_score

      if zip_model == 'Claude2':
        print(f"Scores found - NoOm: {no_om_score}; NoAd: {no_ad_score}; Gram: {gram_score}; Flu: {flu_score}; Coh: {coh_score}.")
      else:
        print(f"Scores found - Gram: {gram_score}; Flu: {flu_score}; NoOm: {no_om_score}; NoAd: {no_ad_score}.")
      print('')

  return formatted_scores, eval_missing, wrong_score

print(f'Loaded evaluations found in {path_dir_unzipped}')
print(f'Model ID used inside evaluation files: {model_prefix}')
eval_files = glob.glob(os.path.join(path_dir_unzipped, '*'))
evals_missing = []
wrong_scores = []
all_scores = []
for filepath in eval_files:
  print(filepath)
  sys0_to_update = None
  if re.search('_inputs85to100_', filepath):
    _, tail = os.path.split(filepath)
    filename = os.path.splitext(tail)[0]
    sys0_to_update = filename.rsplit('_', 1)[1]
    print(f'Will update system ID to {sys0_to_update}')
  dbfile_x = open(filepath, 'rb')
  formatted_scores, eval_missing, wrong_score = loadDataPoint(dbfile_x, model_prefix, sys0_to_update)
  if eval_missing != None:
    evals_missing.append(eval_missing)
  if wrong_score != None:
    wrong_scores.append(wrong_score)
  dbfile_x.close()
  if len(formatted_scores) > 0:
    all_scores.append(formatted_scores)
print(f'Missing evaluations: {evals_missing}')
print(f'Wrong scores: {wrong_scores}')

# Save all scores into a json file
path_json_out = zip_language+'_'+zip_model+'_scores.json'
with codecs.open(path_json_out, 'w', 'utf-8') as outfile:
  json.dump(all_scores, outfile)

In [ ]:
#@title Calculate system_level scores D2T assigned by each LLM for each size

import json
import glob
import os
import math
import pandas as pd

DEBUG = True #@param {type:"boolean"}
update_params_unzip = True #@param {type:"boolean"}
if update_params_unzip:
  zip_model = 'Claude2' #@param['GPT-o3', 'Claude', 'Claude2', 'Llama-13B', 'R1_Llama-70B', 'R1_Llama-70B_bis']

if not os.path.exists('/content/LLM-eval_final_jsons'):
  assert os.path.exists('/content/LLM-eval_final_jsons.zip'), 'You need to upload LLM-eval_final_jsons.zip'
  ! unzip /content/LLM-eval_final_jsons.zip


csv_sys_level_scores = '/content/csv_sysLevel_scores'
if not os.path.exists(csv_sys_level_scores):
  os.makedirs(csv_sys_level_scores)

pd.set_option('display.max_rows', 10)
allowed_scores = [1, 2, 3, 4, 5, 6, 7]
# group_by_size = False#@param{type:'boolean'}
# Store here input sizes for making subpopulations of the test set; should contain input_id:size_category
# input_id starts from 0000
dico_sizes = {}
if zip_model == 'Claude2':
  dico_sizes = {'0570': '0', '0088': '0', '0246': '0', '0213': '1', '0456': '1', '0348': '1', '0652': '2', '0064': '2', '0332': '2', '0479': '3', '0000': '3', '0303': '3', '0025': '4', '0137': '4', '0141': '4', '0147': '5', '0052': '5', '0091': '5', '0126': '6', '0165': '6', '0074': '6', '0117': '7', '0124': '7', '0013': '7', '0076': '8', '0068': '8', '0034': '8', '0144': '9', '0053': '9', '0140': '9'}
else:
  dico_sizes = {'0015': '5' ,'0049': '5' ,'0050': '2' ,'0070': '4' ,'0090': '6' ,'0097': '6' ,'0112': '0' ,'0120': '4' ,'0130': '2' ,'0143': '3' ,'0148': '5' ,'0158': '6' ,'0159': '4' ,'0163': '3' ,'0243': '3' ,'0255': '1' ,'0273': '2' ,'0286': '1' ,'0374': '1' ,'0378': '0' ,'0532': '0'}
# Use this dictionary to map values to other values
score_map = {0:0}

def check_scores(score_list, allowed_scores):
  new_score_list = []
  for score in score_list:
    if score in score_map:
      score = score_map[score]
    assert score in allowed_scores, f'Score out of range: {score}'
    new_score_list.append(score)
  return pd.Series(new_score_list)

# Load the files one at a time; there is one file per LLM-language combination (4*3 = 12 files in total)
json_folder = '/content/LLM-eval_final_jsons'
json_files = glob.glob(os.path.join(json_folder, '*.json'))

pd_list = [pd.read_json(json_file) for json_file in sorted(json_files)]

# Create dataframes to store system-level scores

sysLevel_scores_perLLM = None
if zip_model == 'Claude2':
  sysLevel_scores_perLLM = pd.DataFrame(columns=['language', 'task', 'system', 'input_size', 'LLM', 'avg_no-omissions', 'avg_no-additions', 'avg_grammaticality', 'avg_fluency', 'avg_coherence'])
else:
  sysLevel_scores_perLLM = pd.DataFrame(columns=['language', 'task', 'system', 'input_size', 'LLM', 'avg_no-omissions', 'avg_no-additions', 'avg_grammaticality', 'avg_fluency'])

system_task_counter = 0
for df in pd_list:
  if DEBUG:
    print(df)
  # LLM and Human files use different IDs (eid VS id)
  # split eid column using underscores and create corresponding columns in the dataframe
  df[['language', 'task', 'id', 'system']] = df['eid'].str.split('_', expand=True)

  # NEW Add a column to df with the size of the input according to dico_sizes
  df['input_size'] = df['id'].map(dico_sizes)

  # Convert 'task' and 'system' columns to string to avoid TypeError during sorting
  df['task'] = df['task'].astype(str)
  df['system'] = df['system'].astype(str)

  print('Getting scores for', df['annotator_id'][0], df['language'][0], '...')

  # Get all possible values for task and system column
  unique_tasks = sorted(df['task'].unique())
  unique_systems = sorted(df['system'].unique())
  unique_input_sizes = sorted(df['input_size'].unique())
  print(' Unique tasks:', unique_tasks)
  print(' Unique systems:', unique_systems)
  print(' Unique input sizes:', unique_input_sizes)

  # Get the average scores for each system for each task
  for task in unique_tasks:
    if DEBUG:
      print(f'Task: {task}\n===========')
    for system in unique_systems:
      if DEBUG:
        print(f'\nSystem: {system}')
      for size in unique_input_sizes:
        # get scores in each of the 4 score columns
        no_omissions_scores = df.loc[(df['task'] == task) & (df['system'] == system) & (df['input_size'] == size), 'no-omissions']
        no_omissions_scores = check_scores(no_omissions_scores, allowed_scores)
        no_additions_scores = ''
        # LLM and human files have different criterion names (hyphen VS underscore)
        no_additions_scores = df.loc[(df['task'] == task) & (df['system'] == system) & (df['input_size'] == size), 'no-additions']
        no_additions_scores = check_scores(no_additions_scores, allowed_scores)
        grammaticality_scores = df.loc[(df['task'] == task) & (df['system'] == system) & (df['input_size'] == size), 'grammaticality']
        grammaticality_scores = check_scores(grammaticality_scores, allowed_scores)
        fluency_scores = df.loc[(df['task'] == task) & (df['system'] == system) & (df['input_size'] == size), 'fluency']
        fluency_scores = check_scores(fluency_scores, allowed_scores)
        #Get average scores
        average_no_omissions_score = no_omissions_scores.mean()
        average_no_additions_score = no_additions_scores.mean()
        average_grammaticality_score = grammaticality_scores.mean()
        average_fluency_score = fluency_scores.mean()
        average_coherence_score = 0
        if zip_model == 'Claude2':
          coherence_scores = df.loc[(df['task'] == task) & (df['system'] == system) & (df['input_size'] == size), 'coherence']
          coherence_scores = check_scores(coherence_scores, allowed_scores)
          average_coherence_score = coherence_scores.mean()

        if DEBUG:
          if len(no_omissions_scores) > 0:
            print(f'  Size = {size}')
            print(f'    Average no-omissions score: {average_no_omissions_score}')
            print(f'    Average no-additions score: {average_no_additions_score}')
            print(f'    Average grammaticality score: {average_grammaticality_score}')
            print(f'    Average fluency score: {average_fluency_score}')
            if zip_model == 'Claude2':
              print(f'    Average Coherence score: {average_coherence_score}')

        # If there are no numbers to average, "mean" returns a float that prints 'nan'. Filter out these cases.
        if not math.isnan(average_fluency_score):
          # Add a row to sysLevel_scores_perLLM dataframe
          if zip_model == 'Claude2':
            sysLevel_scores_perLLM.loc[system_task_counter] = [df['language'][0], task, system, size, df['annotator_id'][0], average_no_omissions_score, average_no_additions_score, average_grammaticality_score, average_fluency_score, average_coherence_score]
          else:
            sysLevel_scores_perLLM.loc[system_task_counter] = [df['language'][0], task, system, size, df['annotator_id'][0], average_no_omissions_score, average_no_additions_score, average_grammaticality_score, average_fluency_score]
          system_task_counter += 1

print()
print(sysLevel_scores_perLLM)
# Dump dataframe in a CSV file
sysLevel_scores_perLLM.to_csv(os.path.join(csv_sys_level_scores, 'sysLevel_scores_perLLM_perSize.csv'), index=False)

In [ ]:
#@title Get average (across LLMs) system-level scores D2T and create CSV
# There are 4 LLMs which gave evaluations for all datapoints; we need this extra cell for LLMs to get an average LLM score for each system

DEBUG = True

# Create dataframe to store average LLM-assigned system-level scores
avg_sysLevel_scores_LLMs = None
if zip_model == 'Claude2':
  avg_sysLevel_scores_LLMs = pd.DataFrame(columns=['language', 'task', 'system', 'input_size', 'LLM', 'avg_no-omissions', 'avg_no-additions', 'avg_grammaticality', 'avg_fluency', 'avg_coherence'])
else:
  avg_sysLevel_scores_LLMs = pd.DataFrame(columns=['language', 'task', 'system', 'input_size', 'LLM', 'avg_no-omissions', 'avg_no-additions', 'avg_grammaticality', 'avg_fluency'])

# Now add a row that averages all LLM scores for each system for each task
unique_languages = sorted(sysLevel_scores_perLLM['language'].unique())
unique_tasks_avg = sorted(sysLevel_scores_perLLM['task'].unique())
unique_systems_avg = sorted(sysLevel_scores_perLLM['system'].unique())
unique_input_sizes_avg = sorted(sysLevel_scores_perLLM['input_size'].unique())

# print(unique_languages)
# print(unique_tasks_avg)
# print(unique_systems_avg)

language_system_task_counter = 0
for language in unique_languages:
  if DEBUG:
    print(f'Language: {language}')
  for task in unique_tasks_avg:
    if DEBUG:
      print(f'Task: {task}')
    for system in unique_systems_avg:
      if DEBUG:
        print(f'System: {system}')
      for size in unique_input_sizes_avg:
        if DEBUG:
          print(f'Size: {size}')
        # Get the average scores for each system for each task
        no_omissions_scores = sysLevel_scores_perLLM.loc[(sysLevel_scores_perLLM['language'] == language) & (sysLevel_scores_perLLM['task'] == task) & (sysLevel_scores_perLLM['system'] == system) & (sysLevel_scores_perLLM['input_size'] == size), 'avg_no-omissions']
        no_additions_scores = sysLevel_scores_perLLM.loc[(sysLevel_scores_perLLM['language'] == language) & (sysLevel_scores_perLLM['task'] == task) & (sysLevel_scores_perLLM['system'] == system) & (sysLevel_scores_perLLM['input_size'] == size), 'avg_no-additions']
        grammaticality_scores = sysLevel_scores_perLLM.loc[(sysLevel_scores_perLLM['language'] == language) & (sysLevel_scores_perLLM['task'] == task) & (sysLevel_scores_perLLM['system'] == system) & (sysLevel_scores_perLLM['input_size'] == size), 'avg_grammaticality']
        fluency_scores = sysLevel_scores_perLLM.loc[(sysLevel_scores_perLLM['language'] == language) & (sysLevel_scores_perLLM['task'] == task) & (sysLevel_scores_perLLM['system'] == system) & (sysLevel_scores_perLLM['input_size'] == size), 'avg_fluency']
        # Get average
        average_no_omissions_score = no_omissions_scores.mean()
        average_no_additions_score = no_additions_scores.mean()
        average_grammaticality_score = grammaticality_scores.mean()
        average_fluency_score = fluency_scores.mean()
        average_coherence_score = 0
        if zip_model == 'Claude2':
          coherence_scores = sysLevel_scores_perLLM.loc[(sysLevel_scores_perLLM['language'] == language) & (sysLevel_scores_perLLM['task'] == task) & (sysLevel_scores_perLLM['system'] == system) & (sysLevel_scores_perLLM['input_size'] == size), 'avg_coherence']
          average_coherence_score = coherence_scores.mean()
        if DEBUG:
          if len(no_omissions_scores) > 0:
            print(f'  Average no-omissions score: {average_no_omissions_score}')
            print(f'  Average no-additions score: {average_no_additions_score}')
            print(f'  Average grammaticality score: {average_grammaticality_score}')
            print(f'  Average fluency score: {average_fluency_score}')
            if zip_model == 'Claude2':
              print(f'  Average Coherence score: {average_coherence_score}')

        # If there are no numbers to average, "mean" returns a float that prints 'nan'. Filter out these cases.
        if not math.isnan(average_fluency_score):
          # Add a row to avg_scores dataframe
          if zip_model == 'Claude2':
            avg_sysLevel_scores_LLMs.loc[language_system_task_counter] = [language, task, system, size, 'Average_LLM', average_no_omissions_score, average_no_additions_score, average_grammaticality_score, average_fluency_score, average_coherence_score]
          else:
            avg_sysLevel_scores_LLMs.loc[language_system_task_counter] = [language, task, system, size, 'Average_LLM', average_no_omissions_score, average_no_additions_score, average_grammaticality_score, average_fluency_score]
          language_system_task_counter += 1
          # Dump dataframe in a CSV file
          avg_sysLevel_scores_LLMs.to_csv(os.path.join(csv_sys_level_scores, 'sysLevel_scores_avgLLM_perSize.csv'), index=False)

pd.set_option('display.max_rows', None)
print()
print(avg_sysLevel_scores_LLMs)

In [ ]:
#@title Get average across all sizes

perllmORavgllm = 'perLLM'#@param['perLLM', 'avgLLM']
update_params_unzip = True #@param {type:"boolean"}
if update_params_unzip:
  zip_model = 'Claude2' #@param['GPT-o3', 'Claude', 'Claude2', 'Llama-13B', 'R1_Llama-70B', 'R1_Llama-70B_bis']

path_csv_scores = '/content/csv_sysLevel_scores/sysLevel_scores_'+perllmORavgllm+'_perSize.csv'
df_scores = pd.read_csv(path_csv_scores)
# Get average grouping by same value in the "system" column
# print(df_scores)

sysLevel_scores_LLM = None
if zip_model == 'Claude2':
  sysLevel_scores_LLM = pd.DataFrame(columns=['language', 'task', 'system', 'LLM', 'avg_no-omissions', 'avg_no-additions', 'avg_grammaticality', 'avg_fluency', 'avg_coherence'])
else:
  sysLevel_scores_LLM = pd.DataFrame(columns=['language', 'task', 'system', 'LLM', 'avg_no-omissions', 'avg_no-additions', 'avg_grammaticality', 'avg_fluency'])

system_task_counter = 0
# Get all possible values for task and system column
unique_languages = sorted(df_scores['language'].unique())
unique_tasks = sorted(df_scores['task'].unique())
unique_systems = sorted(df_scores['system'].unique())
unique_input_sizes = sorted(df_scores['input_size'].unique())
print(' Unique tasks:', unique_tasks)
print(' Unique systems:', unique_systems)
print(' Unique input sizes:', unique_input_sizes)
for language in unique_languages:
  for task in unique_tasks:
    for system in unique_systems:
      # get scores in each of the 5 score columns
      no_omissions_scores = df_scores.loc[(df_scores['language'] == language) & (df_scores['task'] == task) & (df_scores['system'] == system), 'avg_no-omissions']
      no_additions_scores = df_scores.loc[(df_scores['language'] == language) & (df_scores['task'] == task) & (df_scores['system'] == system), 'avg_no-additions']
      grammaticality_scores = df_scores.loc[(df_scores['language'] == language) & (df_scores['task'] == task) & (df_scores['system'] == system), 'avg_grammaticality']
      fluency_scores = df_scores.loc[(df_scores['language'] == language) & (df_scores['task'] == task) & (df_scores['system'] == system), 'avg_fluency']
      #Get average scores
      average_no_omissions_score = no_omissions_scores.mean()
      average_no_additions_score = no_additions_scores.mean()
      average_grammaticality_score = grammaticality_scores.mean()
      average_fluency_score = fluency_scores.mean()
      average_coherence_score = 0
      if zip_model == 'Claude2':
        coherence_scores = df_scores.loc[(df_scores['language'] == language) & (df_scores['task'] == task) & (df_scores['system'] == system), 'avg_coherence']
        average_coherence_score = coherence_scores.mean()

      # If there are no numbers to average, "mean" returns a float that prints 'nan'. Filter out these cases.
      if not math.isnan(average_fluency_score):
        # Add a row to sysLevel_scores_perLLM dataframe
        if zip_model == 'Claude2':
          sysLevel_scores_LLM.loc[system_task_counter] = [language, task, system, df_scores['LLM'][0], average_no_omissions_score, average_no_additions_score, average_grammaticality_score, average_fluency_score, average_coherence_score]
        else:
          sysLevel_scores_LLM.loc[system_task_counter] = [language, task, system, df_scores['LLM'][0], average_no_omissions_score, average_no_additions_score, average_grammaticality_score, average_fluency_score]
        system_task_counter += 1

print()
print(sysLevel_scores_LLM)
# Dump dataframe in a CSV file
sysLevel_scores_LLM.to_csv(os.path.join(csv_sys_level_scores, 'sysLevel_scores_'+perllmORavgllm+'_avgSize.csv'), index=False)


In [ ]:
#@title Zip and download csv folders
from google.colab import files

! zip -r /content/csv_sysLevel_scores.zip /content/csv_sysLevel_scores
files.download('/content/csv_sysLevel_scores.zip')

In [ ]:
#@title Compare 2 jsons with results
import json

list_dicos_1 = json.load(open('/content/EN_Claude_scores.json'))
list_dicos_2 = json.load(open('/content/EN_Claude_scores (2).json'))

list1 = sorted(list_dicos_1, key=lambda d: d['eid'])
print(list1)
list2 = sorted(list_dicos_2, key=lambda d: d['eid'])
print(list2)

def list_of_dicts_equal(a: list[dict], b: list[dict]) -> bool:
  return a == b

list_of_dicts_equal(list1, list2)

# Calculate system-level scores humans
Code taken and adapted from the GEM 2024 code


In [ ]:
#@title Calculate system_level scores D2T assigned by the set of human annotators for each size and create CSVs

import json
import glob
import os
import math
import pandas as pd

DEBUG = False

csv_sys_level_scores = '/content/csv_sysLevel_scores'
if not os.path.exists(csv_sys_level_scores):
  os.makedirs(csv_sys_level_scores)

pd.set_option('display.max_rows', 10)
allowed_scores = [1, 2, 3, 4, 5]
dico_sizes = {'0570': '0', '0088': '0', '0246': '0', '0213': '1', '0456': '1', '0348': '1', '0652': '2', '0064': '2', '0332': '2', '0479': '3', '0000': '3', '0303': '3', '0025': '4', '0137': '4', '0141': '4', '0147': '5', '0052': '5', '0091': '5', '0126': '6', '0165': '6', '0074': '6', '0117': '7', '0124': '7', '0013': '7', '0076': '8', '0068': '8', '0034': '8', '0144': '9', '0053': '9', '0140': '9'}
# Use this dictionary to map values to other values
score_map = {0:0}

def check_scores(score_list, allowed_scores):
  new_score_list = []
  for score in score_list:
    if score in score_map:
      score = score_map[score]
    assert score in allowed_scores, f'Score out of range: {score}'
    new_score_list.append(score)
  return pd.Series(new_score_list)

# Load the files one at a time; there is one file per language (since human annotators did not each do all annotations, unlike LLMs)
json_folder = '/content/human-eval_final_jsons'
json_files = glob.glob(os.path.join(json_folder, '*.json'))

pd_list = [pd.read_json(json_file) for json_file in sorted(json_files)]

# Create dataframes to store system-level scores
sysLevel_scores_Humans = pd.DataFrame(columns=['language', 'task', 'system', 'input_size', 'Human', 'avg_grammaticality', 'avg_fluency', 'avg_coherence'])

system_task_counter = 0
for df in pd_list:
  if DEBUG:
    print(df)
  # split id column using underscores and create corresponding columns in the dataframe
  df[['language', 'task', 'id', 'system']] = df['id'].str.split('_', expand=True)
  # NEW Add a column to df with the size of the input according to dico_sizes
  df['input_size'] = df['id'].map(dico_sizes)

  # Convert 'task' and 'system' columns to string to avoid TypeError during sorting
  df['task'] = df['task'].astype(str)
  df['system'] = df['system'].astype(str)

  print('Getting human scores for', df['language'][0])

  # Get all possible values for task and system column
  unique_tasks = sorted(df['task'].unique())
  unique_systems = sorted(df['system'].unique())
  unique_input_sizes = sorted(df['input_size'].unique())
  print(' Unique tasks:', unique_tasks)
  print(' Unique systems:', unique_systems)
  print(' Unique input sizes:', unique_input_sizes)

  # Get the average scores for each system for each task
  for task in unique_tasks:
    if DEBUG:
      print(f'Task: {task}\n===========')
    for system in unique_systems:
      if DEBUG:
        print(f'\nSystem: {system}')
      for size in unique_input_sizes:
        if DEBUG:
          print(f'Size = {size}')
        # get scores in each of the 4 score columns
        grammaticality_scores = df.loc[(df['task'] == task) & (df['system'] == system) & (df['input_size'] == size), 'grammaticality']
        grammaticality_scores = check_scores(grammaticality_scores, allowed_scores)
        fluency_scores = df.loc[(df['task'] == task) & (df['system'] == system) & (df['input_size'] == size), 'fluency']
        fluency_scores = check_scores(fluency_scores, allowed_scores)
        coherence_scores = df.loc[(df['task'] == task) & (df['system'] == system) & (df['input_size'] == size), 'coherence']
        coherence_scores = check_scores(coherence_scores, allowed_scores)
        #Get average scores
        average_grammaticality_score = grammaticality_scores.mean()
        average_fluency_score = fluency_scores.mean()
        average_coherence_score = coherence_scores.mean()
        if DEBUG:
          if len(no_omissions_scores) > 0:
            print(f'  Average grammaticality score: {average_grammaticality_score}')
            print(f'  Average fluency score: {average_fluency_score}')
            print(f'  Average coherence score: {average_coherence_score}')

        # If there are no numbers to average, "mean" returns a float that prints 'nan'. Filter out these cases.
        if not math.isnan(average_fluency_score):
          assert coherence_scores.count() == grammaticality_scores.count() == fluency_scores.count() , 'There should be the same number of ratings for each criterion!'
          # Add row to sysLevel_scores_Humans dataframe
          sysLevel_scores_Humans.loc[system_task_counter] = [df['language'][0], task, system, size, 'Human', average_grammaticality_score, average_fluency_score, average_coherence_score]
          system_task_counter += 1

print()
# At this point, all submissions, including the withdrawn one, were evaluated
# EN = 46 rows: 7 systems * 6 datasets + 1 system * 3 datasets + human *1 dataset
# ES = 18 rows: 3 systems * 6 datasets
# SW = 15 rows: 2 systems * 6 datasets + 1 system * 3 datasets
# All languages = 79 rows; 4 LLMS-> 316 rows
# assert len(sysLevel_scores_Humans) == 64, 'There should be 64 rows (EN+ES) with scores for the human evals!'
# print('Number of rows OK (64).')
print(sysLevel_scores_Humans)
# Dump dataframe in a CSV file
sysLevel_scores_Humans.to_csv(os.path.join(csv_sys_level_scores, 'sysLevel_scores_Humans_perSize.csv'), index=False)

In [ ]:
#@title Get average across all sizes


path_csv_scores = '/content/csv_sysLevel_scores/sysLevel_scores_Humans_perSize.csv'
df_scores = pd.read_csv(path_csv_scores)
# Get average grouping by same value in the "system" column
# print(df_scores)

sysLevel_scores_hum = pd.DataFrame(columns=['language', 'task', 'system', 'Human', 'avg_grammaticality', 'avg_fluency', 'avg_coherence'])

system_task_counter = 0
# Get all possible values for task and system column
unique_languages = sorted(df_scores['language'].unique())
unique_tasks = sorted(df_scores['task'].unique())
unique_systems = sorted(df_scores['system'].unique())
unique_input_sizes = sorted(df_scores['input_size'].unique())
print(' Unique tasks:', unique_tasks)
print(' Unique systems:', unique_systems)
print(' Unique input sizes:', unique_input_sizes)
for language in unique_languages:
  for task in unique_tasks:
    for system in unique_systems:
      # get scores in each of the 5 score columns
      grammaticality_scores = df_scores.loc[(df_scores['language'] == language) & (df_scores['task'] == task) & (df_scores['system'] == system), 'avg_grammaticality']
      fluency_scores = df_scores.loc[(df_scores['language'] == language) & (df_scores['task'] == task) & (df_scores['system'] == system), 'avg_fluency']
      coherence_scores = df_scores.loc[(df_scores['language'] == language) & (df_scores['task'] == task) & (df_scores['system'] == system), 'avg_coherence']
      #Get average scores
      average_grammaticality_score = grammaticality_scores.mean()
      average_fluency_score = fluency_scores.mean()
      average_coherence_score = 0
      average_coherence_score = coherence_scores.mean()

      # If there are no numbers to average, "mean" returns a float that prints 'nan'. Filter out these cases.
      if not math.isnan(average_fluency_score):
        # Add a row to sysLevel_scores_perLLM dataframe
        sysLevel_scores_hum.loc[system_task_counter] = [language, task, system, df_scores['Human'][0], average_grammaticality_score, average_fluency_score, average_coherence_score]
        system_task_counter += 1

print()
print(sysLevel_scores_hum)
# Dump dataframe in a CSV file
sysLevel_scores_hum.to_csv(os.path.join(csv_sys_level_scores, 'sysLevel_scores_Humans_avgSize.csv'), index=False)


# Create plots and LaTeX tables from scores aggregated in "Qualitative evaluation" above.

In [ ]:
#@title Functions from GPT

# Re-import essentials
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
if not os.path.exists('/content/figures'):
  os.mkdir('/content/figures')

update_params_unzip = True #@param {type:"boolean"}
if update_params_unzip:
  zip_model = 'Claude2' #@param['GPT-o3', 'Claude', 'Claude2', 'Llama-13B', 'R1_Llama-70B', 'R1_Llama-70B_bis']

exclude_truncated_outputs = False #@param {type:"boolean"}
exclude_outputs_no_truncated_counterpart = True #@param {type:"boolean"}
systems_to_exclude = []
if exclude_truncated_outputs:
  systems_to_exclude.extend(['e2e-0.95', 'e2e-0.90', 'e2e-0.85', 'Hum-0.9', 'Hum-0.7', 'Hum-0.5'])
if exclude_outputs_no_truncated_counterpart:
  systems_to_exclude.extend(['sng', 'mul', 'Qwen3-32b (EN)', 'Llama2-13b (EN)', 'GPT4.1', 'R1-Llama-70b (EN)', 'Claude3.7-sonnet', 'FORGe', 'UCCIX (GA)'])

# Load the CSV again
file_path = "/content/csv_sysLevel_scores/sysLevel_scores_avgLLM.csv"
df = pd.read_csv(file_path)

# Provided system_IDnames_dico
system_IDnames_dico = {}

if zip_model == 'Claude2':
  system_IDnames_dico = {
    'en': {'00': 'e2e', '01': 'sng', '02': 'mul', '03': 'e2e-0.95', '04': 'e2e-0.90',
           '05': 'e2e-0.85', '06': 'e2e-input0.95', '07': 'e2e-input0.90', '08': 'e2e-input0.85'},
    'ga': {'00': 'e2e', '01': 'sng', '02': 'mul', '03': 'e2e-0.95', '04': 'e2e-0.90',
           '05': 'e2e-0.85', '06': 'e2e-input0.95', '07': 'e2e-input0.90', '08': 'e2e-input0.85'},
  }
else:
  system_IDnames_dico = {
    'en': {'01': 'Qwen3-32b (EN)', '02': 'Llama2-13b (EN)', '03': 'GPT4.1', '04': 'R1-Llama-70b (EN)',
           '05': 'Claude3.7-sonnet', '07': 'FORGe', '08': 'Hum-1.0', '09': 'Hum-0.9',
           '10': 'Hum-0.7', '11': 'Hum-0.5'},
    'ga': {'03': 'GPT4.1', '05': 'Claude3.7-sonnet', '06': 'UCCIX (GA)', '07': 'FORGe',
           '08': 'Hum-1.0', '09': 'Hum-0.9', '10': 'Hum-0.7', '11': 'Hum-0.5'}
  }

# Map system IDs to names based on language
def map_system_name(row):
  lang = row['language']
  sys_id = f"{int(row['system']):02d}"
  return system_IDnames_dico.get(lang, {}).get(sys_id, sys_id)

df['system_name'] = df.apply(map_system_name, axis=1)

# Map input_size indices to bins
size_bins = []
if zip_model == 'Claude2':
  size_bins = ['8-19', '20-39', '40-59', '60-79', '80-99', '100-119', '120-139', '140-159', '160-179', '180-199']
else:
  size_bins = ['8-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69']
df['input_size_bin'] = df['input_size'].map(lambda x: size_bins[int(x)] if int(x) < len(size_bins) else str(x))

value_vars_list = []
if zip_model == 'Claude2':
  value_vars_list=['avg_no-omissions', 'avg_no-additions', 'avg_grammaticality', 'avg_fluency', 'avg_coherence']
else:
  value_vars_list=['avg_no-omissions', 'avg_no-additions', 'avg_grammaticality', 'avg_fluency']
# Melt dataframe to long format for seaborn plotting
df_long = df.melt(
    id_vars=['language', 'system_name', 'input_size_bin'],
    value_vars = value_vars_list,
    var_name='criterion',
    value_name='score'
)

# Prepare consistent color palette across plots for systems
all_systems = sorted(set(df_long['system_name']))
palette = sns.color_palette("tab10", n_colors=len(all_systems))
system_colors = dict(zip(all_systems, palette))

# Plot generation: 4 criteria x 2 languages = 8 plots
criteria_map = {}
max_score = '7'
if zip_model == 'Claude2':
  max_score = '5'
  criteria_map = {
    'avg_no-omissions': 'No-Omissions',
    'avg_no-additions': 'No-Additions',
    'avg_grammaticality': 'Grammaticality',
    'avg_fluency': 'Fluency',
    'avg_coherence': 'Coherence'
  	}
else:
  criteria_map = {
    'avg_no-omissions': 'No-Omissions',
    'avg_no-additions': 'No-Additions',
    'avg_grammaticality': 'Grammaticality',
    'avg_fluency': 'Fluency'
  }

# Create a figure with 4 rows x 2 columns
fig, axes = plt.subplots(len(criteria_map.keys()), 2, figsize=(14, 18), sharex=True, sharey=True)
fig.subplots_adjust(hspace=0.3, wspace=0.15)

handles_list = []
labels_list = []
for i, (crit_key, crit_name) in enumerate(criteria_map.items()):
    for j, lang in enumerate(['en', 'ga']):
        ax = axes[i, j]
        subset = df_long[(df_long['criterion'] == crit_key) & (df_long['language'] == lang)]
        systems_to_process = [system for system in sorted(subset['system_name'].unique()) if system not in systems_to_exclude]
        for system in systems_to_process:
            sys_data = subset[subset['system_name'] == system]
            ax.plot(
                sys_data['input_size_bin'],
                sys_data['score'],
                label=system,
                marker='o',
                color=system_colors[system]
            )
        ax.set_title(f"{crit_name} - {'English' if lang == 'en' else 'Irish'}", fontsize=13)
        ax.set_ylabel(f'Score (1-{max_score})', fontsize=11)
        ax.set_ylim(1, int(max_score))
        ax.grid(True, linestyle=':', alpha=0.4)
        if i == 3:
            ax.set_xlabel('Input Size', fontsize=11)

        handles, labels = ax.get_legend_handles_labels()
        for handle in handles:
            if handle not in handles_list:
                handles_list.append(handle)
        for label in labels:
            if label not in labels_list:
                labels_list.append(label)

# Add a legend
# Single legend outside
# Choose colors of labels
# fig.legend(handles_list, labels_list, title='Systems', bbox_to_anchor=(1.02, 0.5), loc='center left')
fig.legend(handles_list, labels_list, title='Systems', bbox_to_anchor=(0.45, 0.48), loc='lower center')

# plt.suptitle('System Performance Across Criteria, Input Sizes, and Languages', fontsize=16)
plt.tight_layout(rect=[0, 0, 0.88, 0.97])
# Save image in a png file
plt.savefig('/content/figures/system_performance_across_criteria_input_sizes_and_languages.png')
plt.show()


In [ ]:
from google.colab import files

# Download image
! zip -r /content/figures.zip /content/figures
files.download('/content/figures.zip')

# Run divesity metrics on output texts (use T4 GPU Runtime)

In [ ]:
#@title Load file with texts of one language for all systems

import json
import codecs
import re

# Open /content/en_longInputD2T_LLMtexts.json
language = 'en' #@param['en', 'ga']
data = json.load(codecs.open(f'/content/{language}_longInputD2T_LLMtexts.json', 'r', 'utf-8'))

unique_sys_ids_dico = {}
for i, datapoint in enumerate(data):
  dp_id = datapoint['id'].rsplit('_', 1)[1]
  # print(dp_id)
  if dp_id not in unique_sys_ids_dico:
    unique_sys_ids_dico[dp_id] = [datapoint['output']]
  else:
    unique_sys_ids_dico[dp_id].append(datapoint['output'])
  # print(datapoint['text'])

# print(unique_sys_ids)
for key in sorted(unique_sys_ids_dico.keys()):
  print(key, unique_sys_ids_dico[key])

In [ ]:
#@title Diversity metrics functions
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
from tqdm import tqdm
from nltk.util import ngrams
from typing import List, Optional
from collections import Counter
import torch, gc
from sklearn.metrics.pairwise import pairwise_distances_chunked

from diversity import (
    compression_ratio,
    homogenization_score,
    ngram_diversity_score#,
    # self_repetition_score
)

import math
from collections import Counter

# Can't import chamfer_dist from "diversity", so copied it here
# Also added code to flush memory before returning score
def chamfer_dist(
      data: List[str],
      model: Optional[str] = 'Qwen/Qwen3-Embedding-0.6B',
      verbose: Optional[bool] = True,
      batch_size: Optional[int] = 64
) -> float:
  """
  Calculates the chamfer distance for a set of documents (corpus-level).
  This is the average minimum pairwise distance of a data instance to other instances.
  Args:
      data (List[str]): Strings to score.
      model(str, optional): Model to use for embedding. Defaults to 'Qwen/Qwen3-Embedding-0.6B'.
      verbose(bool, optional): Whether to display progress bar. Defaults to True.
      batch_size(int, optional): Batch size for embedding. Defaults to 64.
  Returns:
      float: Chamfer distance.
  """
  model = SentenceTransformer(model)
  embeddings = model.encode(data, batch_size=batch_size, show_progress_bar=verbose)
  distances = cosine_distances(embeddings)
  min_distances = np.min(distances + np.eye(len(distances)) * 1e9, axis=1)

  # Free up memory
  del embeddings, distances, model
  gc.collect()
  if torch.cuda.is_available():
      torch.cuda.empty_cache()
  return np.mean(min_distances).round(3)

# Functions to avoid OOM errors with very long texts (some have up to 6k words and this produces problems when embedding)
def embed_long_text(text, model, max_length=512, overlap=50):
    """Split long text into chunks, embed, then average-pool."""
    tokens = text.split()
    chunks = []
    for i in range(0, len(tokens), max_length - overlap):
        chunk = " ".join(tokens[i:i + max_length])
        chunks.append(chunk)
    with torch.no_grad():
        chunk_embs = model.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
    return np.mean(chunk_embs, axis=0)  # mean pooling

def chamfer_dist_large(
    data,
    model_name='Qwen/Qwen3-Embedding-0.6B',
    batch_size=64,
    verbose=True
):
    model = SentenceTransformer(model_name)

    # 🔹 Embed each document safely (with progress bar)
    embeddings = []
    for doc in tqdm(data, desc="Encoding documents", disable=not verbose):
        emb = embed_long_text(doc, model)
        embeddings.append(emb)
    embeddings = np.vstack(embeddings)

    # 🔹 Compute pairwise distances in chunks (with progress bar)
    min_dists = []
    for chunk in tqdm(
        pairwise_distances_chunked(embeddings, embeddings, metric="cosine"),
        desc="Computing pairwise distances",
        disable=not verbose
    ):
        np.fill_diagonal(chunk, 1e9)
        min_dists.extend(np.min(chunk, axis=1))

    # 🔹 Cleanup
    del embeddings, model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    result = float(np.mean(min_dists))
    # print(f"Chamfer distance: {result:.4f}")
    return result


# Using here because the GitHub version ignores the "n" parameter; changed the following "4" by an "n"
# ngram_docs = [list(set([' '.join(ngram) for ngram in ngrams(doc.split(), 4)])) for doc in dataset]
# ngram_docs = [list(set([' '.join(ngram) for ngram in ngrams(doc.split(), n)])) for doc in dataset]
def self_repetition_score(
        dataset: List[str],
        n: int = 4,
        verbose: bool = True
) -> float:
    """
    Calculates a self-repetition score for a dataset based on the
    repetition of ngrams within the corpus.

    Args:
        dataset (List[str]): A list of documents (strings) to analyze.
        n (int): Size of the ngrams to check for repetition. Defaults to 4.
        verbose (bool): enable/disable show progress bar

    Returns:
        float: The self-repetition score, averaged over the dataset.
    """
    total_sum = 0

    # Get all unique ngrams per doc
    ngram_docs = [list(set([' '.join(ngram) for ngram in ngrams(doc.split(), n)])) for doc in dataset]

    # Count occurrences of unique ngrams across whole dataset
    all_ngrams = sum(ngram_docs, [])
    ngram_counts = Counter(all_ngrams)

    for ngram_doc in tqdm(ngram_docs, desc="Calculating self-repetition score", disable=(not verbose)):
        # Find the total occurrence of an n-gram and subtract current doc's n-gram count
        # to get the count of occurrences of an n-gram in other docs
        sum_ni = sum([ngram_counts[ngram] for ngram in ngram_doc]) - len(ngram_doc)

        # add-one to avoid zero error
        total_sum += np.log(sum_ni + 1)
    return total_sum / len(dataset)

###############################################
# Average text length in characters
def avg_text_length(texts):
  avg_length_c = sum(len(text) for text in texts) / len(texts)
  avg_length_c_2f = round(avg_length_c, 2)
  print(f"Average text length: {avg_length_c:.2f} characters")
  # Average text length in words
  avg_length_w = sum(len(text.split()) for text in texts) / len(texts)
  avg_length_w_2f = round(avg_length_w, 2)
  print(f"Average text length: {avg_length_w:.2f} words")
  print('-----------')
  return avg_length_c_2f, avg_length_w_2f

###############################################
# N-gram diversity (distinct-k); number of unique information units (distinct n-grams) divided by the total number of information units (all n-grams) (https://github.com/cshaib/diversity_)
# Calculated using all n-grams until the indicated max size in the parameter
def ngram_diversity_score_1to4(texts):
  # ngd1 = ngram_diversity_score(texts, num_n=1)
  # ngd1_4f = round(ngd1, 4)
  # print(f"1-gram Diversity: {ngd1:.4f}")
  # ngd2 = ngram_diversity_score(texts, num_n=2)
  # ngd2_4f = round(ngd2, 4)
  # print(f"1-2-gram Diversity: {ngd2:.4f}")
  # ngd3 = ngram_diversity_score(texts, num_n=3)
  # ngd3_4f = round(ngd3, 4)
  # print(f"1-2-3-gram Diversity: {ngd3:.4f}")
  ngd4 = ngram_diversity_score(texts, num_n=4)
  ngd4_4f = round(ngd4, 4)
  print(f"n-gram Diversity (1 to 4): {ngd4:.4f}")
  print('-----------')
  return ngd4_4f

###############################################
# Self repetition (https://github.com/cshaib/diversity)
# Calculates a self-repetition score for a dataset based on the repetition of ngrams within the corpus.
def self_repetition_score_1to4(texts):
  srs1 = self_repetition_score(texts, 1)
  srs1_4f = round(srs1, 4)
  print(f"Unigram Self-repetition: {srs1:.4f}")
  srs2 = self_repetition_score(texts, 2)
  srs2_4f = round(srs2, 4)
  print(f"Bigram Self-repetition: {srs2:.4f}")
  srs3 = self_repetition_score(texts, 3)
  srs3_4f = round(srs3, 4)
  print(f"Trigram Self-repetition: {srs3:.4f}")
  srs4 = self_repetition_score(texts, 4)
  srs4_4f = round(srs4, 4)
  print(f"Quadrigram Self-repetition: {srs4:.4f}")
  print('-----------')
  return srs1_4f, srs2_4f, srs3_4f, srs4_4f

###############################################
# Homogenization score: provides a scores representing how homogeneous a set of documents is (https://github.com/cshaib/diversity)
# (used to compare e.g. texts written with and without assistance of LLMs, showing that texts written using LLMs are more homogeneous, i.e. have a higher Homogenization score).
# metric = 'bleu'
def homogenization_score_metric(texts, metric='rougel', use_stemmer = False):
  hs = homogenization_score(texts, measure=metric, use_stemmer=use_stemmer)
  hs_4f = round(hs, 4)
  print(f"Homogenization): {hs:.4f}")
  print('-----------')
  return hs_4f

###############################################
# Compression ratio (https://github.com/cshaib/diversity)
# Compression ratio calculates the ratio of the size of the compressed file to its original size. If the compression ratio is high, it indicates the file was highly compressible and thus had higher redundancy. This would indicate lower diversity in the file contents.
def compression_ratio_func(texts):
  cr = compression_ratio(texts)
  cr_4f = round(cr, 4)
  print(f"Compression ratio: {cr:.4f}")
  print('-----------')
  return cr_4f

###############################################
# Chamfer dist (https://github.com/cshaib/diversity)
# Calculates the chamfer distance for a set of documents (corpus-level). This is the average minimum pairwise distance of a data instance to other instances.
# Qwen/Qwen3-Embedding-0.6B is multilingual and does support Irish
def chamfer_dist_qwen3(texts):
  chd = chamfer_dist(texts, model = 'Qwen/Qwen3-Embedding-0.6B')
  chd_4f = round(chd, 4)
  print(f"Chamfer distance: {chd:.4f}")
  print('-----------')
  return chd_4f

def chamfer_dist_large_qwen3(texts):
  chd = chamfer_dist_large(texts, model_name = 'Qwen/Qwen3-Embedding-0.6B')
  chd_4f = round(chd, 4)
  print(f"Chamfer distance: {chd:.4f}")
  print('-----------')
  return chd_4f

###############################################
# N-gram Entropy (Ent-n) (https://amitness.com/posts/diversity-evals/)
# The intuition behind it is that in an ideal case, all the texts generated from an LLM would be unique and no n-gram would be repeated more than once.
# We can measure this by collecting all the unique bigrams in the text and calculating their count and the relative frequency. This would give us a probability distribution over the bigrams.
# Diversity = Entropy calculated as follows:
# probs = [0.25, 0.25, 0.25, 0.25]
# -sum(p * math.log(p) for p in probs)
def generate_ngrams(words, n: int):
  return [" ".join(words[i : i + n]) for i in range(len(words) - n + 1)]

def ngram_entropy(texts: list[str], n: int = 2) -> float:
  ngrams = []
  for text in texts:
    words = text.split()
    ngrams.extend(generate_ngrams(words, n))

  ngram_counts = Counter(ngrams)
  total_ngrams = sum(ngram_counts.values())

  ngram_frequencies = [count / total_ngrams for ngram, count in ngram_counts.items()]

  entropy = -sum(freq * math.log(freq) for freq in ngram_frequencies)

  return entropy

# print("Unigram entropy:", ngram_entropy(texts, n=1))
# print("Bigram entropy:", ngram_entropy(texts, n=2))
# print("Trigram entropy:", ngram_entropy(texts, n=3))
# print('-----------')


In [ ]:
#@title Run metrics
import pandas as pd

quick_test = False#@param{'type':'boolean'}

# Provided system_IDnames_dico
system_IDnames_dico = {
    'en': {'01': 'Qwen3-32b (EN)', '02': 'Llama2-13b (EN)', '03': 'GPT4.1', '04': 'R1-Llama-70b (EN)',
           '05': 'Claude3.7-sonnet', '07': 'FORGe', '08': 'Hum-1.0', '09': 'Hum-0.9',
           '10': 'Hum-0.7', '11': 'Hum-0.5'},
    'ga': {'03': 'GPT4.1', '05': 'Claude3.7-sonnet', '06': 'UCCIX (GA)', '07': 'FORGe',
           '08': 'Hum-1.0', '09': 'Hum-0.9', '10': 'Hum-0.7', '11': 'Hum-0.5'}
}

if quick_test:
  texts = [
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog again.",
    "Suddenly, the quick brown fox leaps swiftly over the sleeping dog."
  ]
  # Lexical diversity, length
  avg_length_c_2f, avg_length_w_2f = avg_text_length(texts)
  ngd4_4f = ngram_diversity_score_1to4(texts)
  hs_4f = homogenization_score_metric(texts, 'bleu')
  srs1_4f, srs2_4f, srs3_4f, srs4_4f = self_repetition_score_1to4(texts)
  cr_4f = compression_ratio_func(texts)
  # Semantic diversity
  # chd_4f = chamfer_dist_qwen3(texts)
  chd_4f = chamfer_dist_large_qwen3(texts)

else:
  rows = []
  for key in sorted(unique_sys_ids_dico.keys()):
  # key = '2'
    print(f'############################\nCalculating diversity scores for System {key}....')
    texts = unique_sys_ids_dico[key]
    # Lexical diversity, length
    avg_length_c_2f, avg_length_w_2f = avg_text_length(texts)
    ngd4_4f = ngram_diversity_score_1to4(texts)
    # not sure how to interpret homogenization
    hs_4f = homogenization_score_metric(texts, 'rougel', use_stemmer=True)
    srs1_4f, srs2_4f, srs3_4f, srs4_4f = self_repetition_score_1to4(texts)
    cr_4f = compression_ratio_func(texts)
    # Semantic diversity
    chd_4f = chamfer_dist_large_qwen3(texts)
    rows.append({
    "System": system_IDnames_dico[language][key.zfill(2)],
    # "AvgLenC": f"{avg_length_c_2f:.1f}",
    "AvgLenW": f"{avg_length_w_2f:.1f}",
    "NGD-1to4 ($\uparrow$)": f"{ngd4_4f:.2f}",
    "CompRate ($\downarrow$)": f"{cr_4f:.2f}",
    # "Rep1": f"{srs1_4f:.2f}",
    # "Rep2": f"{srs2_4f:.2f}",
    # "Rep3": f"{srs3_4f:.2f}",
    "SelfRep-4 ($\downarrow$)": f"{srs4_4f:.2f}",
    "Homog-RL ($\downarrow$)": f"{hs_4f:.3f}",
    "ChamDist ($\uparrow$)": f"{chd_4f:.2f}"
    })

  df = pd.DataFrame(rows)
  df = df.sort_values(by="System").reset_index(drop=True)
  latex = df.to_latex(index=False, escape=False, column_format="lccccccccccccc")
  print(latex)


# Create CSVs for calculating statistical significance

In [ ]:
#@title Create one file with all scores for each language+task+system+evaluator combination
#These are the files for calculating correlations

import json
import glob
import os
import pandas as pd

if os.path.exists('/content/csv_for_correls.zip'):
  ! rm -rf /content/csv_for_correls.zip

if not os.path.exists('/content/csv_for_correls'):
  os.mkdir('/content/csv_for_correls')

DEBUG = True#@param{'type':'boolean'}
eval_method = 'LLM' #@param ['LLM', 'Human']
# List here all systems that have scores from all evaluators (hum+LLMS) on at least one dataset. Exclude withdrawn system (system 8)
system_IDnames_dico = {
  'en': {'0': 'e2e', '1': 'sng', '2': 'mul', '3': 'e2e-0.95', '4': 'e2e-0.90',
          '5': 'e2e-0.85', '6': 'e2e-input0.95', '7': 'e2e-input0.90', '8': 'e2e-input0.85'},
  'ga': {'0': 'e2e', '1': 'sng', '2': 'mul', '3': 'e2e-0.95', '4': 'e2e-0.90',
          '5': 'e2e-0.85', '6': 'e2e-input0.95', '7': 'e2e-input0.90', '8': 'e2e-input0.85'},
}

json_files = ''
# LLM and Human files use different IDs (eid VS id)
id_label = ''
if eval_method == 'LLM':
  # Load the files one at a time; there is one file per LLM-language combination (4*3 = 12 files in total)
  json_folder = '/content/LLM-eval_final_jsons'
  json_files = glob.glob(os.path.join(json_folder, '*.json'))
  id_label = 'eid'

elif eval_method == 'Human':
  # Load the files one at a time; there is one file per language (since human annotators did not each do all annotations, unlike LLMs)
  json_folder = '/content/human-eval_final_jsons'
  json_files = glob.glob(os.path.join(json_folder, '*.json'))
  id_label = 'id'

pd_list = [pd.read_json(file) for file in sorted(json_files)]

count_systems_no_score = 0
# The beginning of the following is the same as in the cell that calculates system-level scores
for df in pd_list:
  # if DEBUG:
  #   print(df)
  # split eid column using underscores and create corresponding columns in the dataframe
  df[['language', 'task', 'input_id', 'system']] = df[id_label].str.split('_', expand=True)

  unique_languages = sorted(df['language'].unique())
  unique_tasks = sorted(df['task'].unique())
  # id contains the whole info: language, task, input_id, system
  # unique_ids = sorted(df['id'].unique())
  # Exclude withdrawn system
  unique_systems = [x for x in sorted(df['system'].unique()) if x in system_IDnames_dico[df['language'][0]].keys()]
  if DEBUG:
    if eval_method == 'LLM':
      print(df['annotator_id'][0])
    else:
      print('Human')
    print(unique_languages)
    print(unique_tasks)
    print(unique_systems)
  for language in unique_languages:
    if eval_method == 'LLM':
      print(f'Processing {language} {df["annotator_id"][0]}...')
    else:
      print(f'Processing {language} Human...')
    for task in unique_tasks:
      # We need to count unique_inputs at a task level, since in D2T-1 and D2T-2 the sampled inputs are not the same
      unique_input_ids = sorted(df.loc[(df['language'] == language) & (df['task'] == task), 'input_id'].unique())
      print(f'  {task}: # unique input IDs = {len(unique_input_ids)}')
      for system in unique_systems:
        filename = ''
        if eval_method == 'LLM':
          filename = f'{language}_{task}_{system}_{df["annotator_id"][0]}.csv'
        else:
          filename = f'{language}_{task}_{system}_Human.csv'

        # Check if there are results for the combination of language, task and system
        if df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system)].empty:
          if DEBUG:
            print(f'No scores found for {language}-{task}-{system}')
          count_systems_no_score += 1
        else:
          if DEBUG:
            print(f'{language}-{task}-{system} OK')
          # Make a new dataframe with the columns id, no_om_label, no_add_label, gram_label, flu_label and add rows that correspond to language, task and system
          scores_df = ''
          if eval_method == 'LLM':
            scores_df = pd.DataFrame(columns=['id', 'no-omissions', 'no-additions', 'grammaticality', 'fluency', 'coherence', 'num_scores'])
            # For LLM evals, we have on file per LLM/language in which there is one rating for each of the 180 datapoints for all datasets and all systems (except for the few missing ratings from R1)
            # Thus we simply need to put select the 180 ratings of one system on one dataset (the language condition below is not needed but left just in case)
            scores_df['id'] = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system), 'input_id']
            scores_df['no-omissions'] = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system), 'no-omissions']
            scores_df['no-additions'] = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system), 'no-additions']
            scores_df['grammaticality'] = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system), 'grammaticality']
            scores_df['fluency'] = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system), 'fluency']
            scores_df['coherence'] = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system), 'coherence']
            scores_df['num_scores'] = 1
            # Actually, I'd like the LLM and Human files to be aligned for easier processing, so let's sort the rows
            # I could use the same code as for the human files, but the latter is much slower (I checked, the outputs are the same in both cases)
            scores_df = scores_df.sort_values(by=['id']).reset_index(drop=True)
          else:
            scores_df = pd.DataFrame(columns=['id', 'grammaticality', 'fluency', 'coherence', 'num_scores'])
            # For human evals, we have one file per language with 2 to n annotation for each of the 180 datapoints for all datasets and all systems.
            # We need to average these 2 to n annotations so as to get 180 scores per dataset per system per language, as for LLMs
            # For each language/task/system, get the 2 to n scores for each input
            for i, input_id in enumerate(unique_input_ids):
              scores_gram = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system) & (df['input_id'] == input_id), 'grammaticality']
              scores_flu = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system) & (df['input_id'] == input_id), 'fluency']
              scores_coh = df.loc[(df['language'] == language) & (df['task'] == task) & (df['system'] == system) & (df['input_id'] == input_id), 'coherence']
              scores_df.loc[i] = [input_id, scores_gram.mean(), scores_flu.mean(), scores_coh.mean(), scores_gram.count()]

          if DEBUG:
            print(scores_df)

          # Save scores_df in csv_for_correls
          scores_df.to_csv(os.path.join('/content/csv_for_correls', filename), index=False)

if eval_method == 'LLM':
  print(f'Number of combinations of language/dataset/system with no score: {str(count_systems_no_score)} (expected: 44)')
else:
  print(f'Number of combinations of language/dataset/system with no score: {str(count_systems_no_score)} (expected: 0)')

In [ ]:
#@title Zip and download csv folders
from google.colab import files

! zip -r /content/csv_for_correls.zip /content/csv_for_correls
files.download('/content/csv_for_correls.zip')

# ! zip -r /content/csv_sysLevel_scores.zip /content/csv_sysLevel_scores
# files.download('/content/csv_sysLevel_scores.zip')

#Tukey HSD
These cells takes as input a zip with the CSVs created above for correlations (not exactly the zip created by this notebook). The code was adapted from the WebNLG shared task code with the help of J. Sedoc.

In [ ]:
%%capture
!Rscript -e 'install.packages(c("agricolae", "irr", "reshape2", "Hmisc", "corrplot"), repos="https://cloud.r-project.org/")'

In [ ]:
#@title Unzip files
!unzip csv_for_correls.zip

In [ ]:
#@title Select parameters, create empty folders
import os
DATASET = "D2T-1"   #@param["D2T-1","D2T-2","*"]
SUBSET  = "FA"     #@param["FA", "CFA", "FI", "*"]
LANGUAGE = "ga"     #@param["en", "ga"]
EVALUATOR = "human" #@param["human", "llm"]
# --------------------------
if not os.path.exists('/content/tables_hum-llm-separated'):
  os.mkdir('/content/tables_hum-llm-separated')

if not os.path.exists('/content/tables_hum-llm-together'):
  os.mkdir('/content/tables_hum-llm-together')

## Create TeX tables

In [ ]:
#@title Put CSVs together for computing rankings
from glob import glob
import pandas as pd

# Map of system IDs to system names
system_IDnames_dico = {
  'en': {'0': 'e2e', '1': 'sng', '2': 'mul', '3': 'e2e-0.95', '4': 'e2e-0.90',
          '5': 'e2e-0.85', '6': 'e2e-input0.95', '7': 'e2e-input0.90', '8': 'e2e-input0.85'},
  'ga': {'0': 'e2e', '1': 'sng', '2': 'mul', '3': 'e2e-0.95', '4': 'e2e-0.90',
          '5': 'e2e-0.85', '6': 'e2e-input0.95', '7': 'e2e-input0.90', '8': 'e2e-input0.85'},
}

system_ID_datasets = {'0':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      '1':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      '2':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      '3':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      '4':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      '5':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      '6':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      '7':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      '8':['en_D2T-1-FA', 'ga_D2T-1-FA'],
                      }
# path to the files to consider for calculating the rankings
pattern = ''
if EVALUATOR == 'human':
  pattern = f"{LANGUAGE}/{LANGUAGE}_{DATASET}-{SUBSET}_*_Human.csv"
elif EVALUATOR == 'llm':
  # Models are GPT-4o, GPT-o3, Gemini or R1, while humans are "Human", so looking for G or R should be enough here
  pattern = f"{LANGUAGE}/{LANGUAGE}_{DATASET}-{SUBSET}_*_[GRC]*.csv"
# print(pattern)
files_to_consider = glob(pattern)

total_num_annot = 0
items_IAA = []
data = pd.DataFrame()
for fn in sorted(files_to_consider):
  print(fn)
  # Get team ID
  team = fn.split("_")[2]
  # Get subset to use (are we using this afterwards?)
  # data_segment = fn.split("_")[1].split("-")[-1]
  df = pd.read_csv(fn)
  # Check whether a system needs to be considered for a ranking
  dataset_ID_4_rankings = f'{LANGUAGE}_{DATASET}-{SUBSET}'
  if dataset_ID_4_rankings in system_ID_datasets[team]:
    # Get team name from disctionary above
    df['team'] = system_IDnames_dico[LANGUAGE][team]
    # df['type'] = data_segment
    df.rename(columns={'no-additions': 'no_additions',
                      'no-omissions': 'no_omissions'}, inplace=True)
    number_before_duplication = len(df)

    # For each row, get the number in the number of annotators from which the score was calculated; if more than 1, duplicate the row as many times
    for row in df.iterrows():
      num_annot_to_add = (row[1].iloc[-2])-1
      total_num_annot += row[1].iloc[-2]
      if row[1].iloc[-2] > 2:
        items_IAA.append(fn+str(row[1].iloc[-2]))
      # Add other instances of row to the dataframe
      x = 0
      while x < num_annot_to_add:
        df = pd.concat([df, pd.DataFrame([row[1]], columns=df.columns)])
        x = x + 1
    number_after_duplication = len(df)
    # Print original number of scores and final number of scores used for computing the averages
    print(f'  {number_before_duplication} -> {number_after_duplication} scores used after duplicating rows that have several annotators.')
    data = pd.concat([data, df])
  else:
    print(f'  ID {LANGUAGE}_{DATASET}-{SUBSET} not found for team {team}!')
#data
data.to_csv("all_data_final_averaged.csv", index=False)
print(f'\n----------\nTotal number of human ratings: {total_num_annot} per criterion ({total_num_annot*4} individual ratings.)')
# To get the following number, add 'en_*-*' for systems 0 and 1 in system_ID_datasets
print(f'IAA items: {items_IAA}.')

In [ ]:
#@title Import Python variables into R
%load_ext rpy2.ipython
%R -i EVALUATOR
%R -i LANGUAGE
%R -i DATASET
%R -i SUBSET

In [ ]:
#@title LLM ONLY: Get Mean scores, Tukey HSD and create one LaTeX table for each criterion
%%R
library(xtable)
library(ggplot2)
library(agricolae)
library(reshape2)
library(plyr)
library(Hmisc)
library(corrplot)

label_text <- paste(EVALUATOR, LANGUAGE, DATASET, SUBSET, sep="-")
print(label_text)
all_data_avg <- data.frame(read.csv("all_data_final_averaged.csv"))

# We use anova for the sake of handy calculation of means and putting them to latex tables
# significance and groups are calculated later
anova_function <- function(dep_var, df) {
  fit_semantics <- aov(dep_var ~ team, data=df)
  summary(fit_semantics)
  TukeyHSD(fit_semantics)
  tt <- HSD.test(fit_semantics, "team", group=TRUE)
  tt
}
anova_grammar <- anova_function(all_data_avg$grammaticality, all_data_avg)
anova_fluency <- anova_function(all_data_avg$fluency, all_data_avg)
anova_coherence <- anova_function(all_data_avg$coherence, all_data_avg)
anova_no_omissions <- anova_function(all_data_avg$no_omissions, all_data_avg)
anova_no_additions <- anova_function(all_data_avg$no_additions, all_data_avg)

# seen_data = subset(all_data_avg, type == 'seen')
# unseen_data = subset(all_data_avg, type == 'unseen')

# anova_semantics_seen <- anova_function(seen_data$semantics, seen_data)
# anova_grammar_seen <- anova_function(seen_data$grammar, seen_data)
# anova_fluency_seen <- anova_function(seen_data$fluency, seen_data)
# anova_semantics_unseen <- anova_function(unseen_data$semantics, unseen_data)
# anova_grammar_unseen <- anova_function(unseen_data$grammar, unseen_data)
# anova_fluency_unseen <- anova_function(unseen_data$fluency, unseen_data)

# add_commands <- list(
#   pos = list(0),
#   command = "\\small\n"
# )
# Previous block works with the ",add.to.row = add_commands" parameter in the "print" below. But it inserts the \small at the wrong place so I don't use it now.

# generate latex tables

label_noOm = paste("No-omissions", label_text)
label_noOm_table = paste0("tukey-noOM-", label_text)
df_no_omissions <- data.frame(anova_no_omissions$groups)
####### vvv Lines to have the group as superscript to the mean vvv #######
# df_no_omissions$Mean <- sprintf("%.2f$^{%s}$", df_no_omissions$dep_var, df_no_omissions$groups)
# # remove the raw dep_var and groups columns
# df_no_omissions$dep_var <- NULL
# df_no_omissions$groups <- NULL
# colnames(df_no_omissions) <- c("Mean")
####### ^^^ Lines to have the group as superscript to the mean ^^^ #######
####### vvv Lines to have the group as a separate column vvv #######
colnames(df_no_omissions) <- c("Mean", "Group")
df_no_omissions$Group <- toupper(df_no_omissions$Group)
####### ^^^ Lines to have the group as a separate column ^^^ #######
print('no omissions')
print(xtable(df_no_omissions, caption = label_noOm, label = paste0("tab:", label_noOm_table)), include.rownames = TRUE, sanitize.text.function = identity, file = paste0(label_noOm_table, ".tex"))

label_noAd = paste("No-additions", label_text)
label_noAd_table = paste0("tukey-noAD-", label_text)
df_no_additions <- data.frame(anova_no_additions$groups)
####### vvv Lines to have the group as superscript to the mean vvv #######
# df_no_additions$Mean <- sprintf("%.2f$^{%s}$", df_no_additions$dep_var, df_no_additions$groups)
# # remove the raw dep_var and groups columns
# df_no_additions$dep_var <- NULL
# df_no_additions$groups <- NULL
# colnames(df_no_additions) <- c("Mean")
####### ^^^ Lines to have the group as superscript to the mean ^^^ #######
####### vvv Lines to have the group as a separate column vvv #######
colnames(df_no_additions) <- c("Mean", "Group")
df_no_additions$Group <- toupper(df_no_additions$Group)
####### ^^^ Lines to have the group as a separate column ^^^ #######
print('no additions')
print(xtable(df_no_additions, caption = label_noAd, label = paste0("tab:", label_noAd_table)), include.rownames = TRUE, sanitize.text.function = identity, file = paste0(label_noAd_table, ".tex"))

label_gram = paste("Grammaticality", label_text)
label_gram_table = paste0("tukey-gram-", label_text)
df_grammar <- data.frame(anova_grammar$groups)
####### vvv Lines to have the group as superscript to the mean vvv #######
# df_grammar$Mean <- sprintf("%.2f$^{%s}$", df_grammar$dep_var, df_grammar$groups)
# # remove the raw dep_var and groups columns
# df_grammar$dep_var <- NULL
# df_grammar$groups <- NULL
# colnames(df_grammar) <- c("Mean")
####### ^^^ Lines to have the group as superscript to the mean ^^^ #######
####### vvv Lines to have the group as a separate column vvv #######
colnames(df_grammar) <- c("Mean", "Group")
df_grammar$Group <- toupper(df_grammar$Group)
####### ^^^ Lines to have the group as a separate column ^^^ #######
print('grammaticality')
print(xtable(df_grammar, caption = label_gram, label = paste0("tab:", label_gram_table)), include.rownames = TRUE, sanitize.text.function = identity, file = paste0(label_gram_table, ".tex"))

label_flu = paste("Fluency", label_text)
label_flu_table = paste0("tukey-flu-", label_text)
df_fluency <- data.frame(anova_fluency$groups)
####### vvv Lines to have the group as superscript to the mean vvv #######
# df_fluency$Mean <- sprintf("%.2f$^{%s}$", df_fluency$dep_var, df_fluency$groups)
# # remove the raw dep_var and groups columns
# df_fluency$dep_var <- NULL
# df_fluency$groups <- NULL
# colnames(df_fluency) <- c("Mean")
####### ^^^ Lines to have the group as superscript to the mean ^^^ #######
####### vvv Lines to have the group as a separate column vvv #######
colnames(df_fluency) <- c("Mean", "Group")
df_fluency$Group <- toupper(df_fluency$Group)
####### ^^^ Lines to have the group as a separate column ^^^ #######
print('fluency')
print(xtable(df_fluency, caption = label_flu, label = paste0("tab:", label_flu_table)), include.rownames = TRUE, sanitize.text.function = identity, file = paste0(label_flu_table, ".tex"))

label_coh = paste("Coherence", label_text)
label_coh_table = paste0("tukey-coh-", label_text)
df_coherence <- data.frame(anova_coherence$groups)
####### vvv Lines to have the group as superscript to the mean vvv #######
# df_fluency$Mean <- sprintf("%.2f$^{%s}$", df_fluency$dep_var, df_fluency$groups)
# # remove the raw dep_var and groups columns
# df_fluency$dep_var <- NULL
# df_fluency$groups <- NULL
# colnames(df_fluency) <- c("Mean")
####### ^^^ Lines to have the group as superscript to the mean ^^^ #######
####### vvv Lines to have the group as a separate column vvv #######
colnames(df_coherence) <- c("Mean", "Group")
df_coherence$Group <- toupper(df_coherence$Group)
####### ^^^ Lines to have the group as a separate column ^^^ #######
print('coherence')
print(xtable(df_coherence, caption = label_coh, label = paste0("tab:", label_coh_table)), include.rownames = TRUE, sanitize.text.function = identity, file = paste0(label_coh_table, ".tex"))



In [ ]:
#@title HUMAN ONLY: Get Mean scores, Tukey HSD and create one LaTeX table for each criterion
%%R
library(xtable)
library(ggplot2)
library(agricolae)
library(reshape2)
library(plyr)
library(Hmisc)
library(corrplot)

label_text <- paste(EVALUATOR, LANGUAGE, DATASET, SUBSET, sep="-")
print(label_text)
all_data_avg <- data.frame(read.csv("all_data_final_averaged.csv"))

# We use anova for the sake of handy calculation of means and putting them to latex tables
# significance and groups are calculated later
anova_function <- function(dep_var, df) {
  fit_semantics <- aov(dep_var ~ team, data=df)
  summary(fit_semantics)
  TukeyHSD(fit_semantics)
  tt <- HSD.test(fit_semantics, "team", group=TRUE)
  tt
}
anova_grammar <- anova_function(all_data_avg$grammaticality, all_data_avg)
anova_fluency <- anova_function(all_data_avg$fluency, all_data_avg)
anova_coherence <- anova_function(all_data_avg$coherence, all_data_avg)

# seen_data = subset(all_data_avg, type == 'seen')
# unseen_data = subset(all_data_avg, type == 'unseen')

# anova_semantics_seen <- anova_function(seen_data$semantics, seen_data)
# anova_grammar_seen <- anova_function(seen_data$grammar, seen_data)
# anova_fluency_seen <- anova_function(seen_data$fluency, seen_data)
# anova_semantics_unseen <- anova_function(unseen_data$semantics, unseen_data)
# anova_grammar_unseen <- anova_function(unseen_data$grammar, unseen_data)
# anova_fluency_unseen <- anova_function(unseen_data$fluency, unseen_data)

# add_commands <- list(
#   pos = list(0),
#   command = "\\small\n"
# )
# Previous block works with the ",add.to.row = add_commands" parameter in the "print" below. But it inserts the \small at the wrong place so I don't use it now.

# generate latex tables

label_gram = paste("Grammaticality", label_text)
label_gram_table = paste0("tukey-gram-", label_text)
df_grammar <- data.frame(anova_grammar$groups)
####### vvv Lines to have the group as superscript to the mean vvv #######
# df_grammar$Mean <- sprintf("%.2f$^{%s}$", df_grammar$dep_var, df_grammar$groups)
# # remove the raw dep_var and groups columns
# df_grammar$dep_var <- NULL
# df_grammar$groups <- NULL
# colnames(df_grammar) <- c("Mean")
####### ^^^ Lines to have the group as superscript to the mean ^^^ #######
####### vvv Lines to have the group as a separate column vvv #######
colnames(df_grammar) <- c("Mean", "Group")
df_grammar$Group <- toupper(df_grammar$Group)
####### ^^^ Lines to have the group as a separate column ^^^ #######
print('grammaticality')
print(xtable(df_grammar, caption = label_gram, label = paste0("tab:", label_gram_table)), include.rownames = TRUE, sanitize.text.function = identity, file = paste0(label_gram_table, ".tex"))

label_flu = paste("Fluency", label_text)
label_flu_table = paste0("tukey-flu-", label_text)
df_fluency <- data.frame(anova_fluency$groups)
####### vvv Lines to have the group as superscript to the mean vvv #######
# df_fluency$Mean <- sprintf("%.2f$^{%s}$", df_fluency$dep_var, df_fluency$groups)
# # remove the raw dep_var and groups columns
# df_fluency$dep_var <- NULL
# df_fluency$groups <- NULL
# colnames(df_fluency) <- c("Mean")
####### ^^^ Lines to have the group as superscript to the mean ^^^ #######
####### vvv Lines to have the group as a separate column vvv #######
colnames(df_fluency) <- c("Mean", "Group")
df_fluency$Group <- toupper(df_fluency$Group)
####### ^^^ Lines to have the group as a separate column ^^^ #######
print('fluency')
print(xtable(df_fluency, caption = label_flu, label = paste0("tab:", label_flu_table)), include.rownames = TRUE, sanitize.text.function = identity, file = paste0(label_flu_table, ".tex"))

label_coh = paste("Coherence", label_text)
label_coh_table = paste0("tukey-coh-", label_text)
df_coherence <- data.frame(anova_coherence$groups)
####### vvv Lines to have the group as superscript to the mean vvv #######
# df_fluency$Mean <- sprintf("%.2f$^{%s}$", df_fluency$dep_var, df_fluency$groups)
# # remove the raw dep_var and groups columns
# df_fluency$dep_var <- NULL
# df_fluency$groups <- NULL
# colnames(df_fluency) <- c("Mean")
####### ^^^ Lines to have the group as superscript to the mean ^^^ #######
####### vvv Lines to have the group as a separate column vvv #######
colnames(df_coherence) <- c("Mean", "Group")
df_coherence$Group <- toupper(df_coherence$Group)
####### ^^^ Lines to have the group as a separate column ^^^ #######
print('coherence')
print(xtable(df_coherence, caption = label_coh, label = paste0("tab:", label_coh_table)), include.rownames = TRUE, sanitize.text.function = identity, file = paste0(label_coh_table, ".tex"))



In [ ]:
#@title Export variables to Python
%R -o label_noOm_table
%R -o label_noAd_table
%R -o label_gram_table
%R -o label_flu_table
%R -o label_coh_table
%R -o label_text

In [ ]:
#@title Combine tables for 4 criteria into one figure
import re

noOm_tex = label_noOm_table[0]+'.tex'
noAd_tex = label_noAd_table[0]+'.tex'
gram_tex = label_gram_table[0]+'.tex'
flu_tex = label_flu_table[0]+'.tex'
coh_tex = label_coh_table[0]+'.tex'

list_tex_tables = []
if EVALUATOR == 'llm':
  list_tex_tables = [noOm_tex, noAd_tex, gram_tex, flu_tex, coh_tex]#
else:
  list_tex_tables = [gram_tex, flu_tex, coh_tex]

print(list_tex_tables)

combined_lines = []

# Start figure environment
combined_lines.append("\\begin{figure}[!tbh]")
combined_lines.append("\\small")
combined_lines.append("\\centering")

# Loop through tables and insert as subfigures
for i, f in enumerate(list_tex_tables):
    with open(f, "r") as file:
        lines = file.readlines()

    # Remove the table environment lines (\begin{table}...\end{table})
    table_lines = []
    inside_table = False
    subcaption = ''
    for line in lines:
        if line.strip().startswith("\\begin{table"):
            inside_table = True
            continue
        if line.strip().startswith("\\end{table"):
            inside_table = False
            continue
        if line.strip().startswith("\\caption{"):
            subcaption = line.strip()[9:-1]
            continue
        if inside_table or not ("\\begin{table" in line or "\\end{table" in line):
            # Add phantom letters for vetical alignment in final table
            if re.search(' & B[\\sC]', line):
                line = line.replace(' & B', ' & \\phantom{A}B')
            elif re.search(' & C[\\sD]', line):
                line = line.replace(' & C', ' & \\phantom{A}\\phantom{B}C')
            elif re.search(' & D[\\sE]', line):
                line = line.replace(' & D', ' & \\phantom{A}\\phantom{B}\\phantom{C}D')
            elif re.search(' & E[\\sF]', line):
                line = line.replace(' & E', ' & \\phantom{A}\\phantom{B}\\phantom{C}\\phantom{D}E')
            elif re.search(' & F[\\sG]', line):
                line = line.replace(' & F', ' & \\phantom{A}\\phantom{B}\\phantom{C}\\phantom{D}\\phantom{E}F')
            elif re.search(' & G[\\sH]', line):
                line = line.replace(' & G', ' & \\phantom{A}\\phantom{B}\\phantom{C}\\phantom{D}\\phantom{E}\\phantom{F}G')
            elif re.search(' & H[\\sI]', line):
                line = line.replace(' & H', ' & \\phantom{A}\\phantom{B}\\phantom{C}\\phantom{D}\\phantom{E}\\phantom{F}\\phantom{G}H')
            table_lines.append(line.rstrip())

    # Wrap each table in a minipage (50% width) to make 1x4 column
    combined_lines.append("\\begin{minipage}{0.48\\textwidth}")
    combined_lines.append("\\centering")
    combined_lines.append(f"\\subcaption{{{subcaption}}}")  # optional subcaption
    combined_lines.extend(table_lines)
    combined_lines.append("\\end{minipage}")

    combined_lines.append("\\vspace{0.3cm}\n")  # vertical space
    # Add spacing and line break after every 2 tables
    # if (i + 1) % 2 == 0:
    #     combined_lines.append("\\vspace{0.5cm}\n")  # vertical space
    # else:
    #     combined_lines.append("\\hspace{0.02\\textwidth}")  # horizontal space between tables

# Optional: main figure caption
combined_lines.append("\\caption{System rankings "+str(label_text[0])+"}")
combined_lines.append("\\end{figure}")

clean_label = label_text[0].replace('*', '[]')
output_file = os.path.join('/content/tables_hum-llm-separated', f"table_{clean_label}.tex")
# Write to output file
with open(output_file, "w") as out:
    out.write("\n".join(combined_lines))

print(f"Combined 1x4 table figure saved to {output_file}")

## Move individual tables, Zip and download individual and separated hum-llm tables folders

In [ ]:
# zip and download tables folder
from google.colab import files
!zip -r /content/tables_separated.zip /content/tables_hum-llm-separated

files.download("/content/tables_separated.zip")


In [ ]:
# zip and download tables folder
from google.colab import files
import glob
import os

if not os.path.exists('/content/tables_individual'):
  os.mkdir('/content/tables_individual')

list_indiv_tables = glob.glob('/content/*.tex')
# Move all files to tables_individual
for indiv_table in list_indiv_tables:
  !mv $indiv_table /content/tables_individual

list_indiv_tables_newLoc = glob.glob('/content/tables_individual/*.tex')
# Rename all files, replacing asterisk by []
for indiv_table_reloc in list_indiv_tables_newLoc:
  if re.search('\\*', indiv_table_reloc):
    new_name = indiv_table_reloc.replace('*', '[]')
    os.rename(indiv_table_reloc, new_name)

!zip -r /content/tables_individual.zip /content/tables_individual
files.download("/content/tables_individual.zip")

## Combine TeX tables for  hum and llm results for the 4 criteria.
This block can only be run when you have generated individual tables for each criterion for each evaluation method (llm or human). 8 tables are being merged in one here.

In [ ]:
#@title Unzip folder with all individual tables if not created with the cells above
import os

use_uploaded_zip = False#@param{type:'boolean'}

if use_uploaded_zip:
# Unzip /content/tables_tukey_v2_individual.zip to the folder /content/tables_individual
  !unzip /content/tables_tukey_v2_individual.zip

  # Move folder tables_individual up
  !mv /content/content/tables_individual /content/tables_individual
  !rm -rf /content/content

  if not os.path.exists('/content/tables_hum-llm-together'):
    os.mkdir('/content/tables_hum-llm-together')


In [ ]:
#@title Select parameters
import os
DATASET = "D2T-1"   #@param["D2T-1","D2T-2","*"]
SUBSET  = "FA"     #@param["FA", "CFA", "FI", "*"]
LANGUAGE = "en"     #@param["en", "es", "sw"]
# --------------------------

In [ ]:
#@title Combine llm and human tables for 4 criteria into one figure
import re
import statistics

def count_tukey_groupings(list_groups):
    """
    Takes as input a list with all groups found in a table, e.g. ['A', 'A', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'E', 'E', 'E']
    Returns a count of how many times each group is repeated.
    """
    # Create a dico where instances of each group are counted, e.g. for the list above {'A': 2, 'B': 2, 'C': 3, 'D': 2, 'E': 3}
    dico_count_group_instances = {i:list_groups.count(i) for i in list_groups}
    # Get a list of group duplications (each number in dico above minus 1), e.g. [1, 1, 2, 1, 2] for the dico above.
    list_count_repeated = [dico_count_group_instances[i]-1 for i in dico_count_group_instances]
    # Get the sum of repeated groups
    sum_repeated = sum(list_count_repeated)
    return sum_repeated

def removeAsterisk(original_string):
    clean_string = original_string.replace('*', '[]')
    return(clean_string)

noOm_hum_tex = 'tukey-noOM-human-'+LANGUAGE+'-'+DATASET+'-'+SUBSET+'.tex'
noOm_llm_tex = 'tukey-noOM-llm-'+LANGUAGE+'-'+DATASET+'-'+SUBSET+'.tex'
noAd_hum_tex = 'tukey-noAD-human-'+LANGUAGE+'-'+DATASET+'-'+SUBSET+'.tex'
noAd_llm_tex = 'tukey-noAD-llm-'+LANGUAGE+'-'+DATASET+'-'+SUBSET+'.tex'
gram_hum_tex = 'tukey-gram-human-'+LANGUAGE+'-'+DATASET+'-'+SUBSET+'.tex'
gram_llm_tex = 'tukey-gram-llm-'+LANGUAGE+'-'+DATASET+'-'+SUBSET+'.tex'
flu_hum_tex = 'tukey-flu-human-'+LANGUAGE+'-'+DATASET+'-'+SUBSET+'.tex'
flu_llm_tex = 'tukey-flu-llm-'+LANGUAGE+'-'+DATASET+'-'+SUBSET+'.tex'

new_label_text = LANGUAGE+'-'+DATASET+'-'+SUBSET
clean_label = removeAsterisk(new_label_text)

dico_table_names = {'en-*-*':'English overall', 'en-D2T-1-*':'English in-domain data', 'en-D2T-2-*':'English out-of-domain data', 'en-*-FA':'English factual data', 'en-*-CFA':'English counterfactual data', 'en-*-FI':'English fictional data', 'en-D2T-1-FA':'English in-domain factual data', 'es-*-*':'Spanish overall', 'es-D2T-1-*':'Spanish in-domain data', 'es-D2T-2-*':'Spanish out-of-domain data', 'es-*-FA':'Spanish factual data', 'es-*-CFA':'Spanish counterfactual data', 'es-*-FI':'Spanish fictional data'}

# Order matters here because we're going to insert the tables in this order
list_tex_tables = [noOm_hum_tex, noOm_llm_tex, noAd_hum_tex, noAd_llm_tex, gram_hum_tex, gram_llm_tex, flu_hum_tex, flu_llm_tex]

print(list_tex_tables)

combined_lines = []
# Start figure environment
combined_lines.append("\\begin{figure*}[!tbh]")
combined_lines.append("\\small")
combined_lines.append("\\centering")

# Store scores to get mean scores across systems (useful for analysis)
human_scores_noOm = []
human_scores_noAd = []
human_scores_gram = []
human_scores_flu = []
llm_scores_noOm = []
llm_scores_noAd = []
llm_scores_gram = []
llm_scores_flu = []
# Only count groupings overall, to see which eval methods gives the most ties
human_tukey_global = []
llm_tukey_global = []

# Loop through tables and insert as subfigures
for i, f in enumerate(list_tex_tables):
    human_tukey =  []
    llm_tukey = []
    f_path = removeAsterisk(os.path.join('/content', 'tables_individual', f))
    with open(f_path, "r") as file:
        lines = file.readlines()

    # Remove the table environment lines (\begin{table}...\end{table})
    table_lines = []
    inside_table = False
    subcaption = ''
    for line in lines:
        if line.strip().startswith("\\begin{table"):
            inside_table = True
            continue
        if line.strip().startswith("\\end{table"):
            inside_table = False
            continue
        if line.strip().startswith("\\caption{"):
            subcaption = line.strip()[9:-1]
            continue
        if inside_table or not ("\\begin{table" in line or "\\end{table" in line):
            # Get scores and groups
            if re.search('\\&\\s[0-9]\\.[0-9]+\\s\\&', line):
                score = float(re.search('[0-9]\\.[0-9]+', line).group(0))
                tukey = list(re.subn('^.*[0-9]\\.[0-9]+\\s\\&\\s([A-Z]+).*$', '\\g<1>', line.strip())[0])
                # Filter out DCU-ADAPT-modPB for the averages of D2T-1-* EN dataset, so we can compare numbers to D2T-2 (for which we have no DCU-ADAPT-modPB).
                # Only use for generating the means file, not for the tables, in which we do want DCU-ADAPT-modPB to appear.
                # if not (new_label_text == 'en-D2T-1-*' and re.search('DCU-ADAPT-modPB', line)):
                if re.search('-human-', f):
                    human_tukey.extend(tukey)
                    if re.search('-noOM-', f):
                        human_scores_noOm. append(score)
                    elif re.search('-noAD-', f):
                        human_scores_noAd.append(score)
                    elif re.search('-gram-', f):
                        human_scores_gram.append(score)
                    elif re.search('-flu-', f):
                        human_scores_flu.append(score)
                elif re.search('-llm-', f):
                    llm_tukey.extend(tukey)
                    if re.search('-noOM-', f):
                        llm_scores_noOm. append(score)
                    elif re.search('-noAD-', f):
                        llm_scores_noAd.append(score)
                    elif re.search('-gram-', f):
                        llm_scores_gram.append(score)
                    elif re.search('-flu-', f):
                        llm_scores_flu.append(score)

            # Add phantom letters for vetical alignment in final table
            if re.search(' & B[\\sC]', line):
                line = line.replace(' & B', ' & \\phantom{A}B')
            elif re.search(' & C[\\sD]', line):
                line = line.replace(' & C', ' & \\phantom{A}\\phantom{B}C')
            elif re.search(' & D[\\sE]', line):
                line = line.replace(' & D', ' & \\phantom{A}\\phantom{B}\\phantom{C}D')
            elif re.search(' & E[\\sF]', line):
                line = line.replace(' & E', ' & \\phantom{A}\\phantom{B}\\phantom{C}\\phantom{D}E')
            elif re.search(' & F[\\sG]', line):
                line = line.replace(' & F', ' & \\phantom{A}\\phantom{B}\\phantom{C}\\phantom{D}\\phantom{E}F')
            elif re.search(' & G[\\sH]', line):
                line = line.replace(' & G', ' & \\phantom{A}\\phantom{B}\\phantom{C}\\phantom{D}\\phantom{E}\\phantom{F}G')
            elif re.search(' & H[\\sI]', line):
                line = line.replace(' & H', ' & \\phantom{A}\\phantom{B}\\phantom{C}\\phantom{D}\\phantom{E}\\phantom{F}\\phantom{G}H')
            table_lines.append(line.rstrip())

    human_tukey_global.append(count_tukey_groupings(human_tukey))
    llm_tukey_global.append(count_tukey_groupings(llm_tukey))

    # Wrap each table in a minipage (50% width) to make 1x4 column
    combined_lines.append("\\begin{minipage}{0.48\\textwidth}")
    combined_lines.append("\\centering")
    combined_lines.append(f"\\subcaption{{{subcaption}}}")  # optional subcaption
    combined_lines.extend(table_lines)
    combined_lines.append("\\end{minipage}")

    if re.search('-human-', f):
        combined_lines.append("\\hfill")
    elif re.search('-llm-', f):
        combined_lines.append("\\vspace{0.3cm}\n")  # vertical space
    # Add spacing and line break after every 2 tables
    # if (i + 1) % 2 == 0:
    #     combined_lines.append("\\vspace{0.5cm}\n")  # vertical space
    # else:
    #     combined_lines.append("\\hspace{0.02\\textwidth}")  # horizontal space between tables

# Optional: main figure caption
combined_lines.append("\\caption{System rankings for "+str(dico_table_names[new_label_text])+" (left tables: human ratings, right tables: llm ratings) \\label{fig:tukey-"+str(new_label_text)+"}}")
combined_lines.append("\\end{figure*}")

output_file = os.path.join('/content/tables_hum-llm-together', f"table_{clean_label}.tex")
# Write to output file
with open(output_file, "w") as out:
    out.write("\n".join(combined_lines))

print(f"Combined 2x4 table figure saved to {output_file}")

mean_file = os.path.join('/content/tables_hum-llm-together', f"{clean_label}_means.txt")
with open(mean_file, "w") as out:
    out.write(f'{new_label_text} Human No-Omissions mean = {round(statistics.mean(human_scores_noOm), 2)}\n')
    out.write(f'{new_label_text} Human No-Additions mean = {round(statistics.mean(human_scores_noAd), 2)}\n')
    out.write(f'{new_label_text} Human Grammaticality mean = {round(statistics.mean(human_scores_gram), 2)}\n')
    out.write(f'{new_label_text} Human Fluency mean = {round(statistics.mean(human_scores_flu), 2)}\n\n')

    out.write(f'{new_label_text} LLM No-Omissions mean = {round(statistics.mean(llm_scores_noOm), 2)}\n')
    out.write(f'{new_label_text} LLM No-Additions mean = {round(statistics.mean(llm_scores_noAd), 2)}\n')
    out.write(f'{new_label_text} LLM Grammaticality mean = {round(statistics.mean(llm_scores_gram), 2)}\n')
    out.write(f'{new_label_text} LLM Fluency mean = {round(statistics.mean(llm_scores_flu), 2)}\n\n')

    out.write(f'Human Tukey groupings: {str(sum(human_tukey_global))}\n')
    out.write(f'LLM Tukey groupings: {str(sum(llm_tukey_global))}')

## Zip and download combined hum-llm tables folder

In [ ]:
# zip and download tables folder
from google.colab import files
!zip -r /content/tables_combined.zip /content/tables_hum-llm-together

files.download("/content/tables_combined.zip")